In [1]:
import pandas as pd
from pandas import DataFrame

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime
import numpy as np
import time

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train.head(10)

In [3]:
train.loc[train.status == "fail", "status"] = 0
train.loc[train.status == "pass", "status"] = 1

label = LabelEncoder()

train['cr_ID_num'] = train['cr_ID'].str.slice(start=6)
train['cr_ID_num'] = label.fit_transform(train['cr_ID_num'])

# 3. eq_ID 숫자화 => eq_ID_num
train['eq_ID_num'] = train['eq_ID'].str.slice(start=6)
train['eq_ID_num'] = train['eq_ID_num'].astype(int)

# 1. assembly_time 일 추출 => assembly_time_date
train['assembly_time_date'] = train['assembly_time'].str.slice(start=8, stop=10).astype(int)
# 1-1. assembly_time 요일 추출 => assembly_time_day
train['assembly_time_day'] = train['assembly_time'].apply(lambda x : datetime(int(x[0:4]), int(x[5:7]), int(x[8:10])).weekday())

# 2. process_time과 assembly_time 사이 간격 구하기 => period_sec (초단위)
train['assembly_time'] = train['assembly_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
train['process_time'] = train['process_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

train['period'] = train['process_time'] - train['assembly_time']
train['period'] = train['period'].apply(lambda x : x.seconds)
# 2-1. period_sec 구간 나누기 => period_sec_bin_code
# train['period'] = pd.qcut(train['period'].astype(int), 17)
train['period'] = pd.qcut(train['period'].astype(int), 100)
train['period'] = label.fit_transform(train['period'])

# assembly time과 process time 도 변환
now = datetime.now()
train['assembly_time'] = now - train['assembly_time']
train['assembly_time'] = train['assembly_time'].apply(lambda x : x.seconds)
train['assembly_time'] = pd.qcut(train['assembly_time'].astype(int), 500)
train['assembly_time'] = label.fit_transform(train['assembly_time'])

train['process_time'] = now - train['process_time']
train['process_time'] = train['process_time'].apply(lambda x : x.seconds)
train['process_time'] = pd.qcut(train['process_time'].astype(int), 500)
train['process_time'] = label.fit_transform(train['process_time'])


# 3. ramp 구간 나누기
# train['ramp'] = pd.cut(train['ramp'].astype(int), 1200)
train['ramp'] = pd.qcut(train['ramp'].astype(int), 30)
train['ramp'] = label.fit_transform(train['ramp'])

# 4. servo_defect 구간 나누기
# train['servo_defect'] = pd.cut(train['servo_defect'].astype(int), 1000)
# train['servo_defect'] = pd.qcut(train['servo_defect'].astype(int), 6)
train['servo_defect'] = pd.qcut(train['servo_defect'].astype(int), 500)
train['servo_defect'] = label.fit_transform(train['servo_defect'])

# 5. data_defect 구간 나누기
# train['data_defect'] = pd.cut(train['data_defect'].astype(int), 1000)
# train['data_defect'] = pd.qcut(train['data_defect'].astype(int), 10)
train['data_defect'] = pd.qcut(train['data_defect'].astype(int), 500)
train['data_defect'] = label.fit_transform(train['data_defect'])


# train.drop(['ID', 'cr_ID', 'assembly_time', 'eq_ID', 'process_time'], inplace=True, axis=1)
train.drop(['ID', 'cr_ID', 'eq_ID'], inplace=True, axis=1)


train.head(20)

,assembly_time,process_time,ramp,servo_defect,data_defect,status,cr_ID_num,eq_ID_num,assembly_time_date,assembly_time_day,period
0,280,240,26,466,43,1,11,113,9,0,73
1,432,410,25,7,495,1,16,102,10,1,20
2,496,478,22,387,86,1,13,103,6,4,12
3,371,341,14,305,341,1,0,107,8,6,45
4,338,286,0,35,337,1,2,113,9,0,88
5,186,164,17,57,293,1,8,112,10,1,22
6,240,200,11,58,297,1,5,105,8,6,71
7,496,468,14,160,68,1,3,105,11,2,38
8,364,351,14,167,128,1,4,114,8,6,2
9,123,67,26,124,490,0,12,103,9,0,91


In [4]:
label = LabelEncoder()

test['cr_ID_num'] = test['cr_ID'].str.slice(start=6)
test['cr_ID_num'] = label.fit_transform(test['cr_ID_num'])

# 3. eq_ID 숫자화 => eq_ID_num
test['eq_ID_num'] = test['eq_ID'].str.slice(start=6)
test['eq_ID_num'] = test['eq_ID_num'].astype(int)

# 1. assembly_time 일 추출 => assembly_time_date
test['assembly_time_date'] = test['assembly_time'].str.slice(start=8, stop=10).astype(int)
# 1-1. assembly_time 요일 추출 => assembly_time_day
test['assembly_time_day'] = test['assembly_time'].apply(lambda x : datetime(int(x[0:4]), int(x[5:7]), int(x[8:10])).weekday())

# 2. process_time과 assembly_time 사이 간격 구하기 => period_sec (초단위)
test['assembly_time'] = test['assembly_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
test['process_time'] = test['process_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

test['period'] = test['process_time'] - test['assembly_time']
test['period'] = test['period'].apply(lambda x : x.seconds)
# 2-1. period_sec 구간 나누기 => period_sec_bin_code
# test['period'] = pd.qcut(test['period'].astype(int), 17)
test['period'] = pd.qcut(test['period'].astype(int), 100)
test['period'] = label.fit_transform(test['period'])

# assembly time과 process time 도 변환
now = datetime.now()
test['assembly_time'] = now - test['assembly_time']
test['assembly_time'] = test['assembly_time'].apply(lambda x : x.seconds)
test['assembly_time'] = pd.qcut(test['assembly_time'].astype(int), 500)
test['assembly_time'] = label.fit_transform(test['assembly_time'])

test['process_time'] = now - test['process_time']
test['process_time'] = test['process_time'].apply(lambda x : x.seconds)
test['process_time'] = pd.qcut(test['process_time'].astype(int), 500)
test['process_time'] = label.fit_transform(test['process_time'])

# 3. ramp 구간 나누기
test['ramp'] = pd.qcut(test['ramp'].astype(int), 30)
test['ramp'] = label.fit_transform(test['ramp'])

# 4. servo_defect 구간 나누기
# test['servo_defect'] = pd.qcut(test['servo_defect'].astype(int), 6)
test['servo_defect'] = pd.qcut(test['servo_defect'].astype(int), 500)
test['servo_defect'] = label.fit_transform(test['servo_defect'])

# 5. data_defect 구간 나누기
# test['data_defect'] = pd.qcut(test['data_defect'].astype(int), 10)
test['data_defect'] = pd.qcut(test['data_defect'].astype(int), 500)
test['data_defect'] = label.fit_transform(test['data_defect'])


# test.drop(['ID', 'cr_ID', 'assembly_time', 'eq_ID', 'process_time'], inplace=True, axis=1)
test.drop(['ID', 'cr_ID', 'eq_ID'], inplace=True, axis=1)


test.head(20)

train_label = np.ravel(train.status).astype(int) # Make 1D
train.drop(['status'], inplace=True, axis=1)

# 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 구간 나누기 함수
def make_bins(d, col, factor=2):
    rounding = lambda x: np.around(x / factor)
    d[col] = d[col].apply(rounding)
    return d

In [ ]:
sns.barplot(x="period", y="status", data=train);

In [ ]:
sns.barplot(x="assembly_time_date", y="status", data=train);

In [ ]:
sns.barplot(x="assembly_time_day", y="status", data=train);

In [ ]:
sns.barplot(x="process_time_date", y="status", data=train);

In [ ]:
sns.barplot(x="process_time_day", y="status", data=train);

In [ ]:
# t = make_bins(train.copy(True), 'ramp', 1200)
sns.barplot(x="ramp", y="status", data=train);

In [ ]:
# t = make_bins(train.copy(True), 'servo_defect', 1000)
sns.barplot(x="servo_defect", y="status", data=train);

In [ ]:
# t = make_bins(train.copy(True), 'data_defect', 1000)
sns.barplot(x="data_defect", y="status", data=train);

# 케라스 모델 학습

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(16, activation='relu', input_shape=(6,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(train, train_label, epochs=30, batch_size=1, verbose=1)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.utils import shuffle

def train_and_test(model):
    model.fit(train, train_label)
    prediction = model.predict(test)
    accuracy = round(model.score(train, train_label) * 100, 2)
    print("Accuracy : ", accuracy, "%")
    print("Prediction : ", prediction)
    return prediction

In [6]:
# Logistic Regression
log_pred = train_and_test(LogisticRegression())
# # SVM
svm_pred = train_and_test(SVC())
# #kNN
knn_pred_4 = train_and_test(KNeighborsClassifier(n_neighbors = 16))
# Random Forest
rf_pred = train_and_test(RandomForestClassifier(n_estimators=1000))
# DecisionTree
df_pred = train_and_test(tree.DecisionTreeClassifier())
# Navie Bayes
nb_pred = train_and_test(GaussianNB())

C:\Users\LeeJH\.conda\envs\keras\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy :  72.69 %
Prediction :  [1 1 1 ... 1 0 1]
Accuracy :  76.37 %
Prediction :  [1 1 1 ... 1 0 1]
Accuracy :  79.42 %
Prediction :  [1 1 1 ... 1 0 1]
Accuracy :  100.0 %
Prediction :  [1 1 1 ... 1 0 1]
Accuracy :  100.0 %
Prediction :  [1 1 1 ... 1 0 1]
Accuracy :  73.9 %
Prediction :  [1 1 1 ... 1 0 1]


In [7]:
# result = ['status']
result = {'status' : []}
for pred in rf_pred:
    if pred == 0:
        result['status'].append("fail")
    if pred == 1:
        result['status'].append("pass")

df = pd.DataFrame(result)
df.to_csv('predict19.csv', index=False)